In [1]:
#Importar librerías
import math
import numpy as np
import pandas as pd
from sklearn.svm  import SVC
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from IPython.display import HTML, display

In [2]:
#Leer el archivo con los datos
data = pd.read_csv('./data/msd_genre_dataset.csv')
data=data.drop(['artist_name', 'track_id', 'title'], axis=1)
data1=data.loc[data['genre'] == 'jazz and blues']
data2=data.loc[data['genre'] == 'soul and reggae']
data= pd.concat([data1, data2], axis= 0)
print(data)

                 genre  loudness    tempo  time_signature  key  mode  \
48942   jazz and blues   -18.996   89.147               1    0     1   
48943   jazz and blues   -19.347  125.825               4    2     1   
48944   jazz and blues    -9.472  121.707               4    2     1   
48945   jazz and blues   -14.406   83.012               5    0     1   
48946   jazz and blues    -9.289   76.013               4    7     0   
48947   jazz and blues   -10.003  143.751               4    0     0   
48948   jazz and blues   -11.161  122.412               4    0     1   
48949   jazz and blues   -12.785  244.366               4   10     1   
48950   jazz and blues    -5.834  143.886               1    9     0   
48951   jazz and blues   -12.174   74.642               1    2     0   
48952   jazz and blues    -7.535  139.480               3   11     1   
48953   jazz and blues   -11.945  101.866               7    0     1   
48954   jazz and blues   -11.294  122.794               4    7  

Si se utilizan 600 datos para realizar las validaciones y evaluaciones del modelo respectivamente, entonces la confianza con la que el error estimado esté máximo a 0.1 de diferencia del error real es de:

In [3]:
confianza=1-2*math.exp(-2*0.05**2*600)
print(confianza)

0.9004258632642721


Como se puede observar esta confianza es del 90%. Lo cual significa que usar 600 datos es un buen estimativo para estar máximo a 0.1 del error.
Se parten los datos en entrenamiento y test

In [4]:
#La variable dependiente es la última columna, las independientes son las anteriores.
#Se normaliza las independientes y se convierte nuevamente en un DataFrame de Pandas
x= pd.DataFrame.from_records(preprocessing.normalize(data.loc[:, data.columns != 'genre']))
y= data.iloc[:,0:1]
#Se divide el archivo para entrenamiento y test. Se reserven 10000 datos para test
xTrainT, xTest, yTrainT, yTest = train_test_split(x, y, test_size = 600, random_state = 0)
print(xTrainT)

            0         1         2         3         4         5         6   \
3989 -0.007746  0.030671  0.001081  0.001351  0.000270  0.038641  0.004707   
2962 -0.003219  0.024831  0.000703  0.000527  0.000176  0.142768  0.005611   
6885 -0.003161  0.018672  0.001030  0.000000  0.000206  0.033097  0.007638   
8003 -0.003776  0.038427  0.001272  0.000000  0.000000  0.062554  0.013290   
2528 -0.002559  0.017013  0.000765  0.000000  0.000191  0.048355  0.007027   
6877 -0.001435  0.018808  0.000564  0.000000  0.000141  0.024617  0.005608   
7107 -0.001558  0.019494  0.000460  0.000805  0.000115  0.023422  0.004027   
6453 -0.002304  0.016158  0.000799  0.001598  0.000200  0.051060  0.007661   
5386 -0.002765  0.051769  0.001238  0.003096  0.000000  0.068269  0.013342   
5329 -0.001658  0.032536  0.000746  0.001740  0.000249  0.045790  0.011759   
2260 -0.003426  0.054033  0.001448  0.002533  0.000000  0.063492  0.015787   
3913 -0.001795  0.018823  0.000576  0.000863  0.000144  0.034533

In [5]:
#Se concatenan los datos de test
newData= pd.concat([xTrainT,yTrainT], axis = 1)
xTrain, xValidation, yTrain, yValidation = train_test_split(xTrainT, yTrainT, test_size = 600, random_state = 0)
print(yTrain)

                 genre
59314  soul and reggae
57997  soul and reggae
57632  soul and reggae
49412   jazz and blues
56768  soul and reggae
51584   jazz and blues
58633  soul and reggae
56128  soul and reggae
51624   jazz and blues
49693   jazz and blues
51152   jazz and blues
58302  soul and reggae
50952   jazz and blues
49281   jazz and blues
57155  soul and reggae
58631  soul and reggae
50623   jazz and blues
58624  soul and reggae
50886   jazz and blues
52711   jazz and blues
59396  soul and reggae
50578   jazz and blues
57546  soul and reggae
53223   jazz and blues
49311   jazz and blues
55750  soul and reggae
52198   jazz and blues
55885  soul and reggae
58572  soul and reggae
52506   jazz and blues
...                ...
56954  soul and reggae
51118   jazz and blues
49309   jazz and blues
53082   jazz and blues
56950  soul and reggae
51221   jazz and blues
59092  soul and reggae
50148   jazz and blues
50369   jazz and blues
52958   jazz and blues
49712   jazz and blues
50557   jaz

In [6]:
#Función para entrenar un SVM recibe como parámetros el gamma, el cual es un parámetro de la función gauseanna y el parámetro de regularización C
def trainSVM(gamma,C):
    clf = SVC(C=C, gamma=gamma, random_state=1)
    clf.fit(xTrain, yTrain)   
    return [clf,clf.score(xValidation,yValidation)]

In [7]:
#En el primer arreglo se guardarán los errores y en el segundo los SVM entrenados
errorsLayers=[]
svms=[]

In [9]:
#Se iterarán por gammas y C's en un rango logarítmico. Este rango fue hallado empíricamente
gamma_range = np.logspace(-8, 5, 10)
C_range = np.logspace(-1, 7, 10)

In [10]:
#Se itera por los gammas y los C's y se calcula el porcentaje de éxito de cada svm con los datos de validación
for i in gamma_range:
    errorsTemp=[]
    svmsTemp=[]
    for j in C_range:
        svm=trainSVM(i,j)
        svmsTemp.append(svm[0])
        errorsTemp.append(svm[1])
    errorsLayers.append(errorsTemp)
    svms.append(svmsTemp)
errorsLayers=np.array(errorsLayers)
svms=np.array(svms)

D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y,

D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y,

D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y,

D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
print(errorsLayers)

[[0.55666667 0.55666667 0.55666667 0.55666667 0.55666667 0.55666667
  0.55666667 0.56       0.735      0.71166667]
 [0.55666667 0.55666667 0.55666667 0.55666667 0.55666667 0.55666667
  0.73833333 0.75666667 0.76666667 0.77333333]
 [0.55666667 0.55666667 0.55666667 0.55666667 0.73833333 0.76333333
  0.80166667 0.82666667 0.82833333 0.82833333]
 [0.55666667 0.55666667 0.57333333 0.74666667 0.785      0.81666667
  0.83666667 0.84166667 0.84166667 0.84      ]
 [0.55666667 0.74       0.77       0.80666667 0.83       0.83833333
  0.84166667 0.845      0.83833333 0.83833333]
 [0.75666667 0.79833333 0.82666667 0.835      0.845      0.84333333
  0.84666667 0.87       0.88       0.87833333]
 [0.80666667 0.835      0.85       0.86166667 0.88       0.865
  0.835      0.81       0.79333333 0.79333333]
 [0.79333333 0.83166667 0.83       0.81833333 0.81833333 0.81833333
  0.81833333 0.81833333 0.81833333 0.81833333]
 [0.55666667 0.56333333 0.575      0.575      0.575      0.575
  0.575      0.575    

In [12]:
#Algoritmo para encontrar los índices del elemento más grande de un arreglo 2D, en caso de varias ocurrencias retorna la última
def findMaxIndexes(arr):
    maxC=0
    maxIndex=[-1,-1]
    for i in range(len(arr)):
        for j in range(len(arr[0])):
            if arr[i][j] is not None and arr[i][j]>=maxC:
                maxIndex=[i,j]
                maxC=arr[i][j]
    return maxIndex

In [13]:
#Se utiliza el algoritmo anterior para encontrar la red de mejor resultado
[ind1,ind2]=findMaxIndexes(errorsLayers)
print(ind1,ind2)
print(errorsLayers[ind1][ind2])
print(gamma_range[ind1])
print(C_range[ind2])

6 4
0.88
4.641588833612772
359.38136638046257


In [14]:
#Se muestran los resultados en formato de tabla
df=pd.DataFrame(errorsLayers)
display(HTML(df.to_html()))

,0,1,2,3,4,5,6,7,8,9
0,0.556667,0.556667,0.556667,0.556667,0.556667,0.556667,0.556667,0.560000,0.735000,0.711667
1,0.556667,0.556667,0.556667,0.556667,0.556667,0.556667,0.738333,0.756667,0.766667,0.773333
2,0.556667,0.556667,0.556667,0.556667,0.738333,0.763333,0.801667,0.826667,0.828333,0.828333
3,0.556667,0.556667,0.573333,0.746667,0.785000,0.816667,0.836667,0.841667,0.841667,0.840000
4,0.556667,0.740000,0.770000,0.806667,0.830000,0.838333,0.841667,0.845000,0.838333,0.838333
5,0.756667,0.798333,0.826667,0.835000,0.845000,0.843333,0.846667,0.870000,0.880000,0.878333
6,0.806667,0.835000,0.850000,0.861667,0.880000,0.865000,0.835000,0.810000,0.793333,0.793333
7,0.793333,0.831667,0.830000,0.818333,0.818333,0.818333,0.818333,0.818333,0.818333,0.818333
8,0.556667,0.563333,0.575000,0.575000,0.575000,0.575000,0.575000,0.575000,0.575000,0.575000
9,0.556667,0.556667,0.556667,0.556667,0.556667,0.556667,0.556667,0.556667,0.556667,0.556667


En el punto anterior se observa que un svm con 4.64 de gamma y 359.38 de C fue el que obtuvo el mínimo error, esta tasa de éxito se observa que corresponde a 0.88. Esta tasa de éxito es superior a la obtenida con la red neuronal en el reto 5. Esto puede deberse a que el margen ayuda a que más datos sean clasificados correctamente que con la red neuronal

In [15]:
bestNN= svms[ind1][ind2]
#El svm de mejor resultado se entrena con los datos de entrenamiento totales
bestNN.fit(xTrainT, yTrainT)
#Se prueba con los datos de prueba
bestNN.score(xTest,yTest)

D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8566666666666667

Se puede observar que la tasa de éxito de clasificación es de 0.856666. Esta tasa de éxito es menor que la tasa de éxito con menos datos, esto puede deberse a que otros datos conformaban nuevos vectores de soporte que en la red anterior no se tuvieron en cuenta. De acuerdo al cálculo anterior, se podría afirmar que con una confianza del 90% la tasa de éxito mínima es de 0.8466666666666667.

Se puede concluir que el kernel gausseano es un kernel apropiado para este tipo de datos, dado que la red clasifica más de 8 de cada 10 datos de manera correcta. También que en todos los casos el svm obtuvo una tasa de éxito más alta que una red neuronal. Como se mencionó antes, esto puede deberse a que al buscar el máximo márgen el svm consiga clasificar correctamente puntos imprevistos que se sitúan cercanos al márgen, pero que una red neuronal hubiera clasificado de manera incorrecta.